In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import VotingClassifier
from skimage.io import imread
from skimage.transform import resize
import tensorflow as tf
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten

import os
import glob


load data

In [2]:
# def load_data_from_folder(folder_path):
#     images = []
#     labels = []
#     for label in ["HC", "PD"]:
#         label_path = f"{folder_path}/{label}"
#         for image_path in os.listdir(label_path):
#             full_path = f"{label_path}/{image_path}"
#             try:
#                 img = imread(full_path)
#                 # Convert all images to grayscale
#                 if len(img.shape) == 3:
#                     img = np.mean(img, axis=-1)
#                 img = resize(img, (256, 256)).flatten()  # resize and flatten the image
#                 images.append(img)
#                 labels.append(label)
#             except Exception as e:
#                 print(f"Error processing image: {full_path}")
#                 print(f"Error details: {e}")
#                 continue

#     # Debugging step
#     image_shapes = [img.shape for img in images]
#     unique_shapes = set(image_shapes)
#     print(f"Unique image shapes encountered: {unique_shapes}")

#     return np.array(images), np.array(labels)

# # Load data
# drawings_X, drawings_y = load_data_from_folder("drawings")
# speech_X, speech_y = load_data_from_folder("speech")


data processing

In [5]:
def prepare_images(image_paths, target_size=(224, 224)):
    images = [img_to_array(load_img(img_path, target_size=target_size)) for img_path in image_paths]
    return preprocess_input(np.array(images))

drawing_paths = glob.glob("./drawings/**/**/*.png", recursive=True)
speech_paths = glob.glob("./speech/**/**/*.png", recursive=True)

drawings_X_resized = prepare_images(drawing_paths)
speech_X_resized = prepare_images(speech_paths)

# Load the ResNet50 model with weights pre-trained on ImageNet without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

drawings_features = base_model.predict(drawings_X_resized)
speech_features = base_model.predict(speech_X_resized)


118/135 [=========================>....] - ETA: 8s

train model

In [ ]:
def create_cnn(input_shape):
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))  # Assuming 2 classes: HC and PD
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Reshape features to fit into the CNN
drawings_features_reshaped = drawings_features.reshape(drawings_features.shape[0], 1, 1, drawings_features.shape[1])
speech_features_reshaped = speech_features.reshape(speech_features.shape[0], 1, 1, speech_features.shape[1])

# Train the CNN for drawings data
drawings_model = create_cnn((1, 1, 2048))  # 2048 features from ResNet50 with pooling='avg'
drawings_model.fit(drawings_features_reshaped, drawings_y, epochs=10, batch_size=32, validation_split=0.2)

# Train the CNN for speech data
speech_model = create_cnn((1, 1, 2048))
speech_model.fit(speech_features_reshaped, speech_y, epochs=10, batch_size=32, validation_split=0.2)

Performance for Drawings:
              precision    recall  f1-score   support

          HC       0.75      0.93      0.83       267
          PD       0.81      0.50      0.62       165

    accuracy                           0.76       432
   macro avg       0.78      0.71      0.72       432
weighted avg       0.77      0.76      0.75       432


Performance for Speech:
              precision    recall  f1-score   support

          HC       1.00      0.94      0.97        17
          PD       0.96      1.00      0.98        24

    accuracy                           0.98        41
   macro avg       0.98      0.97      0.97        41
weighted avg       0.98      0.98      0.98        41



evaluate, ensemble

In [ ]:
drawings_model.save('drawings_model.h5')
speech_model.save('speech_model.h5')


AttributeError: 'RandomForestClassifier' object has no attribute 'save'